In [1]:
import base64
import io
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import time
from flask import Flask, send_file

# Add Flask to serve the glb file
app_flask = Flask(__name__)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Serve the glb file using Flask
@app_flask.route('/static/3d_model.glb')
def serve_glb_model():
    return send_file('static/3d_model.glb', as_attachment=True)

# Define the layout of the app with the background image
app.layout = html.Div([
    html.Div(
        [
            html.H1("Image to 3D Converter", className="text-center mt-2 display-4", style={"position": "absolute", "top": "45px", "left": "50%", "transform": "translateX(-50%)"}),
            dcc.Upload(
                id="upload-image",
                children=html.Div([
                    "Drag and Drop or ",
                    html.A("Select an Image", className="btn btn-primary", style={"text-align": "center"})
                ]),
                style={
                    "width": "50%",
                    "height": "60px",
                    "lineHeight": "45px",
                    "borderWidth": "0.5px",
                    "borderStyle": "dashed",
                    "borderRadius": "5px",
                    "textAlign": "center",
                    "margin": "10px",
                    "position": "absolute",
                    "top": "110px",
                    "left": "50%",
                    "transform": "translateX(-50%)"
                },
                multiple=False
            ),
             html.Div(style={
                #"background-color": "white",
                "width": "2px",
                "height": "30%",
                "position": "absolute",
                "top": "250px",
                "left": "49%",
                "transform": "translateX(-50%)",
                "borderWidth": "0.5px",
                "borderStyle": "dashed",
                "borderRadius": "5px"
            }),
            html.Div(id="image-container", style={"position": "absolute", "top": "300px", "left": "20%"}),
            dbc.Button("Convert", id="convert-button", color="success", className="mt-3", style={"position": "absolute", "top": "600px", "left": "44%"}),
            dbc.Button("Cancel", id="cancel-button", color="danger", className="mt-3 ml-2", style={"position": "absolute", "top": "600px", "left": "50%"}),
            dcc.Loading(id="loading-output", type="default", color="success"),
            html.Div(id="model-viewer", style={"position": "absolute", "top": "250px", "right": "20%"}),
        ],
        style={
            'background-image': 'url("https://i.postimg.cc/J7DYwHkK/3d.jpg")',
            'background-size': 'cover',
            'background-repeat': 'no-repeat',
            'background-attachment': 'fixed',
            'font-family': 'Arial, sans-serif',
            'color': '#ffffff',
            'text-align': 'center',
            'position': 'relative',
            'padding': '0px',
            'height': '100vh'
        }
    )
])

# Define the callback to handle image uploading and display the image with a white border
@app.callback(Output("image-container", "children"), [Input("upload-image", "contents")]
)
def update_image(contents):
    if contents is not None:
        _, content_string = contents.split(',')
        image_data = base64.b64decode(content_string)
        image_data_base64 = base64.b64encode(image_data).decode('utf-8')
        img_src = f"data:image/jpeg;base64,{image_data_base64}"
        img = html.Img(
            src=img_src,
            style={
                "max-width": "100%",
                "border": "0.5px dashed white",  # White border with specified properties
                "borderRadius": "5px"
            }
        )
        return img
    else:
        return html.Div("Please upload an image.", style={"position": "fixed", "top": "200px", "left": "45%"})


# Define the callback to handle "Cancel" button click and clear the uploaded image
@app.callback(Output("upload-image", "contents"), [Input("cancel-button", "n_clicks")], [State("upload-image", "contents")]
)
def cancel_upload(n_clicks, current_contents):
    if n_clicks is not None and n_clicks > 0:
        return None
    else:
        return current_contents

# Define the callback to show the 3D model after a 10-second delay
@app.callback(Output("model-viewer", "children"), [Input("convert-button", "n_clicks")]
)
def show_3d_model(n_clicks):
    if n_clicks is not None and n_clicks > 0:
        time.sleep(10)  # Simulate a 10-second delay
        # Insert the code to embed your 3D model here (e.g., an iframe with your Sketchfab model)
        model_html = html.Iframe(src="https://sketchfab.com/models/31a0e13f09bc4e7185fd3dd2072aca74/embed", width="120%", height="450", style={"border": "none"})
        return model_html
    else:
        return ""

if __name__ == '__main__':
    app.run_server(debug=True, port=3006, threaded=True)
